In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [60]:
def recommend_similar_games(input_games_str,data_frame, num_recs):
    
    # Convert the comma-separated string to a list of input games
    input_games = [game.strip() for game in input_games_str.split('@@@')]
    #st.write("this is the data input ",input_games)
    games =data_frame
    # Review Ratio is the Number of Positive Reviews made on steam of a particular game divided by the total number of steam reviews for that game
    games["Review Ratio"]=(games["Positive"]/(games["Positive"]+games["Negative"]))
    games["Review Sum"]=(games["Positive"]+games["Negative"])

    # steam_total_reviews is the total number of reviews made for all games on steam
    steam_total_reviews=sum(games['Review Sum'])
    steam_total_reviews

    # Combined Score is the # of positive reviews for a game divided by the total number of reviews for all games on steam
    games['Combined Score'] = games['Review Ratio'] * (games['Review Sum']/steam_total_reviews)
    
    # Preprocess the data
    data_frame['Genres'] = data_frame['Genres'].fillna('')
    data_frame['Combined Score'] = data_frame['Combined Score'].fillna(0)
    data_frame['Tags'] = data_frame['Tags'].fillna('')  # Add this line to handle missing Tags
    data_frame['Features'] = data_frame['Genres'] + ' ' + data_frame['Combined Score'].astype(str) + ' ' + data_frame['Tags']

    # Create a TF-IDF vectorizer for the combined features
    tfidf_vectorizer_combined = TfidfVectorizer()
    tfidf_matrix_combined = tfidf_vectorizer_combined.fit_transform(data_frame['Features'])

    # Build a nearest neighbors model for combined features
    knn_combined = NearestNeighbors(n_neighbors=30, metric='cosine')  # Increase n_neighbors to allow for more recommendations
    knn_combined.fit(tfidf_matrix_combined)

    recommended_games = set()

    for input_game_name in input_games:
        # Find the index of the given game in the DataFrame
        game_index = data_frame[data_frame['Name'] == input_game_name].index[0]

        # Use the nearest neighbors model to find the most similar games based on combined features
        distances_combined, indices_combined = knn_combined.kneighbors(tfidf_matrix_combined[game_index])

        # Get the 6 most similar games based on combined features
        similar_game_indices_combined = indices_combined[0]
        similar_games_combined = [data_frame.loc[i] for i in similar_game_indices_combined]

        # Sort the recommendations by the "Combined Score" in descending order
        similar_games_combined = sorted(similar_games_combined, key=lambda x: x['Combined Score'], reverse=True)

        # Filter out the input games and any duplicates
        unique_recommendations = [game for game in similar_games_combined if
                                game['Name'] != input_game_name and game['Name'] not in input_games and game['Name'] not in recommended_games]

        # Add unique game names to the set
        #recommended_games.update(game['Name'] for game in unique_recommendations[:num_recs])
        
        for i in range(0, num_recs):
            rec_game = unique_recommendations[i]['Name']
            df_score = data_frame.loc[data_frame['Name'] == rec_game] 
            score = df_score['Combined Score'].iloc[0]
            res = (input_game_name, rec_game, score)
            
            recommended_games.add((input_game_name, rec_game, score))
            

    return recommended_games

In [61]:
games = pd.read_csv("games.csv")

In [64]:
# Example usage with three input games (up to 5)
input_games_str = "Galactic Bowling@@@Don't Drop the Bass@@@Christmas Santa Troubles"
recommended_games = recommend_similar_games(input_games_str, games, 6)

for item in recommended_games:
    print(item)

('Galactic Bowling', 'Fastlane Bowling', 1.2111440317553246e-08)
('Galactic Bowling', 'HyperBowl', 4.844576127021298e-07)
('Galactic Bowling', 'VR Mini Bowling 2', 3.633432095265974e-08)
('Christmas Santa Troubles', 'Cosmic Kites', 7.266864190531946e-08)
('Christmas Santa Troubles', 'DUCK CASINO: BULLET', 7.266864190531946e-08)
('Galactic Bowling', 'Knockout Bowling VR', 0.0)
('Christmas Santa Troubles', 'Hot Runback - VR Runner', 7.266864190531946e-08)
('Christmas Santa Troubles', 'Master Pyrox Wizard Smackdown', 7.266864190531946e-08)
("Don't Drop the Bass", 'モールス信号で愛を伝えよ。', 0.0)
('Galactic Bowling', 'Pixel Puzzles Mosaics', 7.266864190531948e-07)
("Don't Drop the Bass", 'Cosmic Kites', 7.266864190531946e-08)
("Don't Drop the Bass", 'Master Pyrox Wizard Smackdown', 7.266864190531946e-08)
('Galactic Bowling', 'Lift It', 7.266864190531948e-07)
("Don't Drop the Bass", 'Bomb!Bomb!Carnival!!', 0.0)
("Don't Drop the Bass", 'DUCK CASINO: BULLET', 7.266864190531946e-08)
('Christmas Santa Tro